In [1]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile
import joblib

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import metrics
from sklearn.metrics import recall_score, precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
make_scorer = sklearn.metrics.make_scorer
f1 = make_scorer(f1_score, pos_label=1, average="binary")
from sklearn.metrics import classification_report
from sklearn.inspection import permutation_importance
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import RepeatedStratifiedKFold

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


In [2]:
## Build XGBoost Model
from copy import deepcopy
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,confusion_matrix,roc_curve
import xgboost as xgb
import time
import warnings

# 1. Data Preparation

In [3]:
trainDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\5ppm\\df_train_dbMSIn5ppm4nonInDI_STDnorm.csv")
diDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\5ppm\\df_nonInDI_dbMSIn5ppm4nonInDI_STDnorm.csv")

In [4]:
trainDEFSDf

,pixel_id,311.1684,269.2486,215.0328,241.2173,295.2278,435.2965,480.3096,883.533,309.1725,339.1996,233.1546,337.2055,265.1478,279.2328,790.5408,303.2328,325.1842,type
0,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,3.688075,0.571627,-1.174372,1.063717,-0.402608,0.530386,-1.162140,-0.460314,4.504229,2.433620,0.763553,4.206358,4.516292,-0.820173,-0.938106,-0.366259,3.236921,0
1,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,5.100656,1.500452,-0.377475,1.371974,-0.402608,0.261448,-1.162140,0.114807,5.622844,3.295467,1.669318,4.660548,5.492306,-0.600716,-1.052497,-0.356656,4.400250,0
2,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,3.446205,0.632002,-0.825142,0.988573,-0.402608,-0.132213,-0.001739,-0.444349,3.715588,2.422229,1.925770,3.145579,4.409094,-0.574920,-1.218949,-0.595259,2.951787,0
3,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,3.197035,0.437932,-0.418539,1.280690,-0.402608,-0.081180,-1.162140,-0.346415,3.504202,2.330939,1.504090,3.809358,4.249205,-0.749250,-0.676211,-0.705813,2.725904,0
4,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,1.997351,0.761102,-1.300546,0.782923,-0.402608,0.628778,-1.162140,-0.335517,2.568640,1.465051,0.493887,2.525056,2.983379,-0.602580,-0.720711,-0.551822,1.668743,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90955,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.410654,-1.035619,0.040891,-1.168013,-0.402608,1.089779,0.894498,-0.912395,0.737794,0.451720,0.701177,-0.408120,0.504213,-1.012988,-0.681860,-0.745363,0.388220,1
90956,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.252531,-1.035619,-0.899617,0.716853,-0.402608,1.349007,-1.162140,-0.397604,0.870632,0.530896,0.443536,-0.408120,0.260001,-1.287821,-0.193146,-0.758145,0.407976,1
90957,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.357592,1.264396,-0.920862,-1.168013,-0.402608,0.878974,0.567022,-0.582258,0.522238,0.493701,1.245964,-0.408120,-0.734974,-1.024784,0.706513,-0.403020,0.412271,1
90958,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.526713,-1.035619,-1.209545,-1.168013,-0.402608,0.671900,0.623847,-1.045097,0.702885,0.357600,0.669866,-0.408120,0.671895,-0.941051,0.038292,-0.377786,0.401027,1


In [6]:
can = []
for i in list(range(0,4)) + list(range(5,10)) + list(range(11,12)) + list(range(15,16)) + [len(list(trainDEFSDf.columns))-1]:
    can.append(list(trainDEFSDf.columns)[i])

dican = []
for i in list(range(0,4)) + list(range(5,10)) + list(range(11,12)) + list(range(15,16)) + [len(list(diDEFSDf.columns))-1]:
    dican.append(list(diDEFSDf.columns)[i])

In [7]:
can

['pixel_id',
 '311.1684',
 '269.2486',
 '215.0328',
 '295.2278',
 '435.2965',
 '480.3096',
 '883.533',
 '309.1725',
 '233.1546',
 '790.5408',
 'type']

In [8]:
dican

['pixel_id',
 '311.1684',
 '269.2486',
 '215.0328',
 '295.2278',
 '435.2965',
 '480.3096',
 '883.533',
 '309.1725',
 '233.1546',
 '790.5408',
 'type']

In [9]:
## input training set ## 90960 x 20 df
trainDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\5ppm\\df_train_dbMSIn5ppm4nonInDI_STDnorm.csv")
#trainDEFSDf[trainDEFSDf.type .== 1, :]
## calculate weight ## 0: 47449, 1: 43511
trainDEFSDf = trainDEFSDf[can]
Yy_train = trainDEFSDf["type"]  # 0.9585; 1.0453
sampleW = []
for w in Yy_train:
    if w == 0:
        sampleW.append(0.9585)
    elif w == 1:
        sampleW.append(1.0453) 


In [10]:
## input ext val set ## 6075 x 20 df
extDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\5ppm\\df_ext_dbMSIn5ppm4nonInDI_STDnorm.csv")
#extDEFSDf[extDEFSDf.type .== 1, :]
## calculate weight ## 0: 2943, 1: 3132
extDEFSDf = extDEFSDf[can]
Yy_ext = extDEFSDf["type"]  # 1.0321; 0.9698
sampleExtW = []
for w in Yy_ext:
    if w == 0:
        sampleExtW.append(1.0321)
    elif w == 1:
        sampleExtW.append(0.9698) 


In [11]:
## input ingested set ## 97035 x 20 df
ingestedDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\5ppm\\df_ingested_dbMSIn5ppm4nonInDI_STDnorm.csv")
#ingestedDEFSDf[extDEFSDf.type .== 1, :]
## calculate weight ## 0: 50392, 1: 46643
ingestedDEFSDf = ingestedDEFSDf[can]
Yy_ingested = ingestedDEFSDf["type"]  # 0.9628; 1.0402
sampleIngestedW = []
for w in Yy_ingested:
    if w == 0:
        sampleIngestedW.append(0.9628)
    elif w == 1:
        sampleIngestedW.append(1.0402)


In [12]:
## input FNA set ## 88701 x 20 df
fnaDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\5ppm\\df_FNA_dbMSIn5ppm4nonInDI_STDnorm.csv")
#fnaDEFSDf[fnaDEFSDf.type .== 1, :]
## calculate weight ##  0: 44540, 1: 44161
fnaDEFSDf = fnaDEFSDf[can]
Yy_FNA = fnaDEFSDf["type"]  # 0.9957; 1.0043
sampleFNAW = []
for w in Yy_FNA:
    if w == 0:
        sampleFNAW.append(0.9957)
    elif w == 1:
        sampleFNAW.append(1.0043)


In [13]:
## input DirectIn set ## 88701 x 20 df
diDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\5ppm\\df_nonInDI_dbMSIn5ppm4nonInDI_STDnorm.csv")
#diDEFSDf[diDEFSDf.type .== 1, :]
## calculate weight ## 0: 3027, 1: 3030
diDEFSDf = diDEFSDf[dican]

Yy_DI = diDEFSDf["type"]  # 1.0007; 0.9993
sampleDiW = []
for w in Yy_DI:
    if w == 0:
        sampleDiW.append(1.0007)
    elif w == 1:
        sampleDiW.append(0.9993)


In [14]:
diDEFSDf

,pixel_id,311.1684,269.2486,215.0328,295.2278,435.2965,480.3096,883.533,309.1725,233.1546,790.5408,type
0,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.402342,0.913244,0.833746,-0.387446,0.518836,1.015027,0.488609,1.697873,2.628182,0.519166,0
1,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.314493,1.539995,0.003972,0.129889,0.742545,0.064314,0.246510,1.609584,2.740049,0.703606,0
2,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.841655,1.554217,0.670368,0.339209,0.796962,0.931195,0.949744,2.182584,3.441873,1.160177,0
3,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.621964,2.090087,0.272553,0.593092,0.596528,0.208169,1.078951,1.903016,2.615834,0.447406,0
4,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.303707,1.234845,0.156936,0.247718,0.913738,0.858728,0.548974,1.677419,3.034290,0.765049,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3023,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,0.046377,0.004282,1.384092,0.640055,0.529506,0.782214,1.053116,-0.172416,0.198570,0.272497,1
3024,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-1.799346,-1.043795,-1.051923,-1.271685,-1.475923,-2.069352,0.599188,-1.114427,-0.133371,1.695744,1
3025,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.519249,-1.667011,1.280725,-0.144515,0.768313,0.241011,0.110271,-0.360029,-0.084657,0.353490,1
3026,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.419602,-0.812576,1.794612,-0.889368,-0.280928,-0.362442,0.994469,-0.357697,0.057898,0.474667,1


In [15]:
## define functions for performace evaluation ##

# Average score
def avgScore(arrAcc, cv):
    sumAcc = 0
    for acc in arrAcc:
        sumAcc += acc
    return sumAcc / cv


In [16]:
trainDEFSDf[trainDEFSDf.columns[1:-1]]

,311.1684,269.2486,215.0328,295.2278,435.2965,480.3096,883.533,309.1725,233.1546,790.5408
0,3.688075,0.571627,-1.174372,-0.402608,0.530386,-1.162140,-0.460314,4.504229,0.763553,-0.938106
1,5.100656,1.500452,-0.377475,-0.402608,0.261448,-1.162140,0.114807,5.622844,1.669318,-1.052497
2,3.446205,0.632002,-0.825142,-0.402608,-0.132213,-0.001739,-0.444349,3.715588,1.925770,-1.218949
3,3.197035,0.437932,-0.418539,-0.402608,-0.081180,-1.162140,-0.346415,3.504202,1.504090,-0.676211
4,1.997351,0.761102,-1.300546,-0.402608,0.628778,-1.162140,-0.335517,2.568640,0.493887,-0.720711
...,...,...,...,...,...,...,...,...,...,...
90955,0.410654,-1.035619,0.040891,-0.402608,1.089779,0.894498,-0.912395,0.737794,0.701177,-0.681860
90956,0.252531,-1.035619,-0.899617,-0.402608,1.349007,-1.162140,-0.397604,0.870632,0.443536,-0.193146
90957,0.357592,1.264396,-0.920862,-0.402608,0.878974,0.567022,-0.582258,0.522238,1.245964,0.706513
90958,0.526713,-1.035619,-1.209545,-0.402608,0.671900,0.623847,-1.045097,0.702885,0.669866,0.038292


In [17]:
import warnings
warnings.filterwarnings('ignore')

# 2. Unsupervised Learning

## 2.1. Dataset Preparation

In [18]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [19]:
df_ROI_for_ML_Opti = pd.concat([trainDEFSDf, extDEFSDf, fnaDEFSDf, diDEFSDf]).set_index('pixel_id')

In [20]:
df_ROI_for_ML_Opti_ingested = ingestedDEFSDf.set_index('pixel_id')

In [21]:
df_ROI_for_ML_Opti_train = trainDEFSDf.set_index('pixel_id')

In [22]:
df_ROI_for_ML_Opti_ext = extDEFSDf.set_index('pixel_id')

In [23]:
df_ROI_for_ML_Opti_FNA = fnaDEFSDf.set_index('pixel_id')

In [24]:
df_ROI_for_ML_Opti_di = diDEFSDf.set_index('pixel_id')

In [25]:
df_ROI_for_ML_Opti_di.describe()

,311.1684,269.2486,215.0328,295.2278,435.2965,480.3096,883.533,309.1725,233.1546,790.5408,type
count,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3028.000000
mean,3.556527e-17,3.668356e-17,5.942700e-16,3.436998e-16,4.651981e-16,-3.780918e-16,-9.191972e-16,-5.811438e-17,-7.931788e-16,-3.019381e-17,0.500330
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.500082
min,-2.776735e+00,-3.684381e+00,-2.720374e+00,-3.328833e+00,-1.475923e+00,-3.366659e+00,-2.320051e+00,-1.701767e+00,-2.178006e+00,-3.705233e+00,0.000000
25%,-7.685900e-01,-6.134711e-01,-5.996852e-01,-5.392095e-01,-5.787979e-01,-6.072195e-01,-4.512491e-01,-6.450113e-01,-4.288067e-01,-6.762906e-01,0.000000
50%,9.775581e-02,-2.030662e-01,1.619175e-02,-2.568702e-02,1.457927e-01,-3.398478e-02,8.422639e-02,-3.576545e-01,-1.809827e-01,-1.353200e-01,1.000000
75%,7.284643e-01,3.619300e-01,6.212090e-01,5.052305e-01,7.469444e-01,6.593097e-01,5.960037e-01,2.576578e-01,4.129057e-02,5.229941e-01,1.000000
max,5.686665e+00,4.275386e+00,4.628362e+00,4.617973e+00,3.235670e+00,3.841278e+00,4.062752e+00,3.719943e+00,4.286845e+00,5.935154e+00,1.000000


## 2.2. PCA Analysis

In [37]:
## PCA Analysis

def pca_visual(df=df_ROI_for_ML_Opti_di):

    pca = PCA(n_components=3)
    raMSI_ML_mz_df = df.drop("type", axis = 1)
    pca.fit(raMSI_ML_mz_df)
    result = pd.DataFrame(pca.transform(raMSI_ML_mz_df), columns=['PCA%i' % i for i in range(3)], index=df_ROI_for_ML_Opti_di.index)
    
    plt.subplots(dpi = 300)
    plt.scatter(result['PCA1'], result['PCA2'], c = df['type'], s=1)
    plt.xlabel('PC1', size=20)
    plt.ylabel('PC2', size=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/dbMSIn_5ppm/PCA_di.tif", bbox_inches = 'tight')

    components = pca.fit_transform(raMSI_ML_mz_df)
    total_var = pca.explained_variance_ratio_.sum() * 100
    fig = px.scatter_3d(
        components, x=0, y=1, z=2, color=df['type'],
        title = f'Total Explained Variance: {total_var:.2f}%',
        labels = {'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
    )
    fig.show()

In [ ]:
pca_visual()

# 3. Supervised Learning

## 3.1. Preparation of training and test dataset

In [26]:
def split_dataset(df=df_ROI_for_ML_Opti_ingested):
    raMSI_ML_NoType = df.drop("type", axis = 1)
    raMSI_ML_Type=df[['type']]
    X_train,X_test,y_train,y_test=train_test_split(raMSI_ML_NoType,raMSI_ML_Type,test_size=0.2,random_state=42)
    return X_train,X_test,y_train,y_test

X_train,X_test,y_train,y_test=split_dataset(df=df_ROI_for_ML_Opti)

In [27]:
X_ingested = df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[:-1]]

In [28]:
y_ingested = df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[-1]]

In [29]:
X_train = df_ROI_for_ML_Opti_train[df_ROI_for_ML_Opti_train.columns[:-1]]

In [30]:
y_train = df_ROI_for_ML_Opti_train[df_ROI_for_ML_Opti_train.columns[-1]]

In [31]:
X_val = df_ROI_for_ML_Opti_ext[df_ROI_for_ML_Opti_ext.columns[:-1]]

In [32]:
y_val = df_ROI_for_ML_Opti_ext[df_ROI_for_ML_Opti_ext.columns[-1]]

In [33]:
X_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[:-1]]

In [34]:
y_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[-1]]

In [35]:
X_di = df_ROI_for_ML_Opti_di[df_ROI_for_ML_Opti_di.columns[:-1]]

In [36]:
y_di = df_ROI_for_ML_Opti_di[df_ROI_for_ML_Opti_di.columns[-1]]

## 3.2 Machine Learning Modeling: Model 1+2 (Train 10:10, Val 4:4, Test 3:3)

In [37]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

### SVM

In [38]:
## Build LinearSVC Model
from sklearn.svm import LinearSVC

def LinearSVC_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, XTest2=X_di, yTrain=y_ingested, yVal=y_val, yTest=y_FNA, yTest2=y_di):
    model_svm = LinearSVC(penalty='l2', loss="hinge", C=402, random_state=42, class_weight={0:0.9628, 1:1.0402})
    model_svm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_svm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_svm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_svm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_svm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_svm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print("FNA Test Set Precision:", metrics.precision_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_svm.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_svm.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    # Test2 set
    predictions_di = model_svm.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, model_svm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_svm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_svm.predict(XTest2)))
    print("DirectIn Set Precision:", precision_score(yTest2, model_svm.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    
    return x, y

In [39]:
coeff_SVM, TOPfeatures_SVM = LinearSVC_func()

Training Set F1: 0.911105817500426
Training Set MCC: 0.8304945644407106
Training Set Accuracy: 0.9157431838170624
              precision    recall  f1-score   support

           0       0.90      0.95      0.92     47449
           1       0.94      0.88      0.91     43511

    accuracy                           0.92     90960
   macro avg       0.92      0.91      0.92     90960
weighted avg       0.92      0.92      0.92     90960

[[45044  2405]
 [ 5259 38252]]
Ext Val Set F1: 0.7714637598356837
Ext Val Set MCC: 0.5316090664087487
Ext Val Set Accuracy: 0.7662551440329218
              precision    recall  f1-score   support

           0       0.77      0.74      0.75      2943
           1       0.76      0.79      0.78      3132

    accuracy                           0.77      6075
   macro avg       0.77      0.77      0.77      6075
weighted avg       0.77      0.77      0.77      6075

[[2175  768]
 [ 652 2480]]
5F-CV: 0.88419358882852
FNA Set F1: 0.7099070265532839
FNA Set

In [40]:
import joblib as jl

In [41]:
model = LinearSVC(penalty='l2', loss="hinge", C=402, random_state=42, class_weight={0:0.9628, 1:1.0402})
model.fit(X_ingested, y_ingested)

LinearSVC(C=402, class_weight={0: 0.9628, 1: 1.0402}, loss='hinge',
          random_state=42)

In [42]:
modelSavePath = r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\modelSVM_dbMSIn5ppm.joblib"
jl.dump(model, modelSavePath, compress = 5)

['I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\raMSIn\\modeling\\modelSVM_dbMSIn5ppm.joblib']

## 3.3 Machine Learning Modeling: Model 1 (Train 6:6, Val 4:4, Test 3:3)

### SVM

In [43]:
## Build LinearSVC Model
from sklearn.svm import LinearSVC

def LinearSVC_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, XTest2=X_di, yTrain=y_train, yVal=y_val, yTest=y_FNA, yTest2=y_di):
    model_svm = LinearSVC(penalty='l2', loss="hinge", C=402, random_state=42, class_weight={0:0.9628, 1:1.0402})
    model_svm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_svm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_svm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_svm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_svm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_svm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print("FNA Test Set Precision:", metrics.precision_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_svm.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_svm.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    # Test2 set
    predictions_di = model_svm.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, model_svm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_svm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_svm.predict(XTest2)))
    print("DirectIn Set Precision:", precision_score(yTest2, model_svm.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    return x, y

In [44]:
coeff_SVM, TOPfeatures_SVM = LinearSVC_func()

Training Set F1: 0.9261972502402819
Training Set MCC: 0.8548685743844722
Training Set Accuracy: 0.9278803869832893
              precision    recall  f1-score   support

           0       0.92      0.94      0.93     47449
           1       0.93      0.91      0.92     43511

    accuracy                           0.93     90960
   macro avg       0.93      0.93      0.93     90960
weighted avg       0.93      0.93      0.93     90960

[[44680  2769]
 [ 3791 39720]]
Ext Val Set F1: 0.7096405268855542
Ext Val Set MCC: 0.4671182328121864
Ext Val Set Accuracy: 0.7287242798353909
              precision    recall  f1-score   support

           0       0.69      0.80      0.74      2943
           1       0.78      0.66      0.71      3132

    accuracy                           0.73      6075
   macro avg       0.73      0.73      0.73      6075
weighted avg       0.74      0.73      0.73      6075

[[2368  575]
 [1073 2059]]
5F-CV: 0.8992905138064954
FNA Set F1: 0.6947394335961404
FNA 

## 3.4 Machine Learning Modeling: Model 2 (Train 4:4, Val 6:6, Test 3:3)

### SVM

In [45]:
## Build LinearSVC Model
from sklearn.svm import LinearSVC

def LinearSVC_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, XTest2=X_di, yTrain=y_val, yVal=y_train, yTest=y_FNA, yTest2=y_di):
    model_svm = LinearSVC(penalty='l2', loss="hinge", C=402, random_state=42, class_weight={0:0.9628, 1:1.0402})
    model_svm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_svm.predict(XVal)
    print("Training Set F1:", f1_score(yVal, model_svm.predict(XVal), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, model_svm.predict(XVal), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_svm.predict(XTrain)
    print("Ext Val Set F1:", f1_score(yTrain, model_svm.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, model_svm.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_svm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_svm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_svm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print("FNA Test Set Precision:", metrics.precision_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_svm.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_svm.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    # Test2 set
    predictions_di = model_svm.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, model_svm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_svm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_svm.predict(XTest2)))
    print("DirectIn Set Precision:", precision_score(yTest2, model_svm.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

    
    return x, y

In [46]:
coeff_SVM, TOPfeatures_SVM = LinearSVC_func()

Training Set F1: 0.7410576080698456
Training Set MCC: 0.43373778325729145
Training Set Accuracy: 0.7058707124010554
              precision    recall  f1-score   support

           0       0.79      0.59      0.68     47449
           1       0.65      0.83      0.73     43511

    accuracy                           0.71     90960
   macro avg       0.72      0.71      0.70     90960
weighted avg       0.72      0.71      0.70     90960

[[28209 19240]
 [ 7514 35997]]
Ext Val Set F1: 0.8681036936631468
Ext Val Set MCC: 0.7657568881042005
Ext Val Set Accuracy: 0.8758847736625515
              precision    recall  f1-score   support

           0       0.82      0.96      0.88      2943
           1       0.95      0.80      0.87      3132

    accuracy                           0.88      6075
   macro avg       0.88      0.88      0.88      6075
weighted avg       0.89      0.88      0.88      6075

[[2812  131]
 [ 623 2509]]
5F-CV: 0.8123424174797422
FNA Set F1: 0.6235509544093579
FNA

# 4. Model Evaluation

## 4.1 Prediction

In [47]:
## deploy models for training set ##
model = jl.load(r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\modelSVM_dbMSIn5ppm.joblib")
#
## deploy model ##
predicted0n1_train = model.predict(X_train)      
trainDEFSDf["predicted0n1"] = predicted0n1_train
T_ = []
F_ = []
for i in range(len(trainDEFSDf)):
    if trainDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif trainDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
trainDEFSDf["T"] = T_
trainDEFSDf["F"] = F_
#
## save ##, ouputing trainSet df
savePath = r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\df_train_dbMSIn5ppm_STDnorm_0n1.csv"
trainDEFSDf.to_csv(savePath)

In [48]:

## deploy model ##
predicted0n1_ext = model.predict(X_val)
extDEFSDf["predicted0n1"] = predicted0n1_ext
T_ = []
F_ = []
for i in range(len(extDEFSDf)):
    if extDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif extDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
extDEFSDf["T"] = T_
extDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\df_ext_dbMSIn5ppm_STDnorm_0n1.csv"
extDEFSDf.to_csv(savePath)

In [49]:

## deploy model ##
predicted0n1_FNA = model.predict(X_FNA)
fnaDEFSDf["predicted0n1"] = predicted0n1_FNA
T_ = []
F_ = []
for i in range(len(fnaDEFSDf)):
    if fnaDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif fnaDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
fnaDEFSDf["T"] = T_
fnaDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\df_FNA_dbMSIn5ppm_STDnorm_0n1.csv"
fnaDEFSDf.to_csv(savePath)

In [50]:

## deploy model ##
predicted0n1_Di = model.predict(X_di)
diDEFSDf["predicted0n1"] = predicted0n1_Di
T_ = []
F_ = []
for i in range(len(diDEFSDf)):
    if diDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif diDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
diDEFSDf["T"] = T_
diDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\df_Di_dbMSIn5ppm_STDnorm_0n1.csv"
diDEFSDf.to_csv(savePath)